In [1]:
def check_info(self, word):
        sem_rgx = re.compile (r'[Q][A-Z]{3}')  # semantic tagset
        syn_rgx = re.compile (r'[Y][A-Z]{3}')  # syntactic tagset
        dom_rgx = re.compile (r'[X]{1}[ABCDEFGHIJKLMNOPQRSTUVWYZ]{3}')  # domain tagset
        ent_rgx = re.compile (r'[X]{2}[A-Z]{2}')  # entity tagset
        mor_rgx = re.compile (r'[A-Z]{3}')  # morph tagset

        if sem_rgx.match (word):
            return 'SenInfo'
        elif syn_rgx.match (word):
            return 'SynInfo'
        elif dom_rgx.match (word):
            return 'DomInfo'
        elif ent_rgx.match (word):
            return 'EntInfo'
        elif mor_rgx.match (word):
            return 'MorInfo'


    def merge(self):

        #들어오는 데이터를 병합할 변수
        global merge_data
        global cnt

        #처음들어왔을 때는 병합할 필요가 없으므로 열린 파일을 merge_data에 저장
        if (self.dataFrame_Tab.currentIndex()-1) == 0:
            merge_data = handle_df_list[0]
        
        #두 번째부터는 들어오는 데이터를 data변수에 저장하고 
        #이전에 저장해둔 merge_data와 data를 concat으로 병합한 뒤
        #두 데이터를 sort_values로 정렬을 시켜준다.
        else:
            data = handle_df_list[self.dataFrame_Tab.currentIndex()-1]
            merge_data = pd.concat([merge_data,data],ignore_index=True)
            merge_data = merge_data.sort_values(by='Lemma')
            merge_data = merge_data.reset_index()
            cnt += 1

        #사용자가 입력할 col_name.
        #gui상에서 .text()로 입력받는다.
        col_data = 'MorInfo1'

        #우선시 되는 데이터 : my_data, 지워져도 되는 데이터 del_data 
        #gui상에서 .text()로 입력받는다.
        my_data = 'ZND'
        del_data = 'ZNZ'

        
        #파일들을 1개 이상 입력 받으면 실행된
        if cnt != 0:
            #중복 단어들을 저장하는 리스트
            reduplication = []
            #i는 처음부터 끝에서 두번째에 있는 단어들까지 돌아가고
            #j는 i번째에 있는 단어 바로 다음 단어랑 비교를 한다.
            #왜냐하면 정렬이 되어있는 상태기 때문에 바로 전 후의 단어들을 비교해서
            #앞글자가 다르면 패스를 해서 시간을 단축시키기 위함이다.
            for i in range(0,len(merge_data)-1):
                for j in range(i+1, i+cnt+1):
                    #first에는 i번째 단어를 음절별로 나누어서 저장하고
                    #second에는 j번째(i다음 단어)를 음절로 나누어서 저장한다.
                    first = Divide(merge_data.loc[i,'Lemma'])
                    second = Divide(merge_data.loc[j,'Lemma'])

                    #저장한 단어들의 앞글자가 다르면 i를 1증가시켜 다음 단어로 넘어간다.
                    if first[0] != second[0]:
                        break
                    
                    #만약 단어의 앞글자가 같다면 아래 코드를 실행한다.
                    elif first[0] == second[0]:
                        #앞글자는 같지만 단어가 다르면 for문을 종료한다.
                        if first != second:
                            break
                        
                        #앞글자가 같고 단어까지 같으면 입력 받은 mydata가 i번째에 있는지 아님 j(i+1)번째 있는지 확인한다.
                        elif first == second:
                            if my_data in merge_data.loc[i,col_data] and my_data in merge_data.loc[j,col_data]:
                                pass
                            elif my_data in merge_data.loc[i,col_data] and del_data in merge_data.loc[j,col_data]:
                                #reduplication.append(merge_data.iloc[i].values.tolist())
                                #reduplication.append(merge_data.iloc[j].values.tolist())

                                # stem_data에는 i번째 단어 정보를 리스트형태로 저장하고
                                # follow_data에는 j번째 (i+1)번째 단어 정보를 리스트 형태로 저장한다.
                                # 변수 x는 follow_data를 돌면서 follow_data요소가 stem_data에 정보가 없으면 
                                # 그 정보들 check_info()함수에 넘겨서 정보를 col에 저장해준다(ex. MorInfo)
                                # y는 merge_data의 colum 정보들을 돌면서
                                # check_info로 입력받은 정보가 들어있는 column이 처음으로 빈칸이 나오는 장소에
                                # stem_data에 들어있지 않은 정보(follow_data)를 merge_data에 넣어준다.
                                stem_data1 = merge_data.iloc[i].values.tolist()
                                follow_data1 = merge_data.iloc[j].values.tolist()
                                for x in range(4, len(follow_data1)-1):
                                    if follow_data1[x] not in stem_data1:
                                        col = self.check_info(follow_data1[x])
                                        col_nme = merge_data.columns
                                        for y in range(4,len(col_nme)):
                                           #print()
                                            if col in col_nme[y] and merge_data.loc[j, col_nme[y]] == '':
                                                merge_data.loc[j, col_nme[y]] = follow_data1[x]
                                                break
                                
                            elif del_data in merge_data.loc[i,col_data] and my_data in merge_data.loc[j,col_data]:
                                #reduplication.append(merge_data.iloc[i].values.tolist())
                                #reduplication.append(merge_data.iloc[j].values.tolist())
                                stem_data2 = merge_data.iloc[j].values.tolist()
                                follow_data2 = merge_data.iloc[i].values.tolist()
                                for x in range(4, len(follow_data2)-1):
                                    if follow_data2[x] not in stem_data2:
                                        col = self.check_info(follow_data2[x])
                                        col_nme = merge_data.columns
                                        for y in range(4,len(col_nme)):
                                           #print()
                                            if col in col_nme[y] and merge_data.loc[j, col_nme[y]] == '':
                                                merge_data.loc[j, col_nme[y]] = follow_data2[x]
                                                break


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 20)

In [7]:
import pandas as pd
import re
import numpy as np

def check_info(word):
    sem_rgx = re.compile (r'[Q][A-Z]{3}')  # semantic tagset
    syn_rgx = re.compile (r'[Y][A-Z]{3}')  # syntactic tagset
    dom_rgx = re.compile (r'[X]{1}[ABCDEFGHIJKLMNOPQRSTUVWYZ]{3}')  # domain tagset
    ent_rgx = re.compile (r'[X]{2}[A-Z]{2}')  # entity tagset
    mor_rgx = re.compile (r'[A-Z]{3}')  # morph tagset

    if sem_rgx.match (word):
        return 'SenInfo'
    elif syn_rgx.match (word):
        return 'SynInfo'
    elif dom_rgx.match (word):
        return 'DomInfo'
    elif ent_rgx.match (word):
        return 'EntInfo'
    elif mor_rgx.match (word):
        return 'MorInfo'

def column_name(df):
    # 첫 행 살리기
    first = list (df.columns)
    if first[0] == 0:
        pass
    else:
        df.loc[0] = first
        for val in first:
            if 'Unnamed' in val:
                x = first.index (val)
                first[x] = np.nan
        df.loc[0] = first
    df = df.fillna ('')
    sem_rgx = re.compile (r'[Q][A-Z]{3}')  # semantic tagset
    syn_rgx = re.compile (r'[Y][A-Z]{3}')  # syntactic tagset
    dom_rgx = re.compile (r'[X]{1}[ABCDEFGHIJKLMNOPQRSTUVWYZ]{3}')  # domain tagset
    ent_rgx = re.compile (r'[X]{2}[A-Z]{2}')  # entity tagset
    mor_rgx = re.compile (r'[A-Z]{3}')  # morph tagset

    # 컬럼의 총 개수를 l에 저장한다.
    # 컬럼의 개수 만큼 lemma와 category뒤에 lemma와 category개수인 2를 뺀만큼
    # ''를 추가해 주어 해당 컬럼 개수 만큼의 리스트 col_nme을 만들어 준다.
    l = len (df.columns)
    col_nme = ['Lemma', 'Category']
    for i in range (l - 2):
        col_nme.append ('')
    # sem =>SemInfo 뒤에 붙을 숫자
    # syn =>SynInfo 뒤에 붙을 숫자
    # dom =>DomInfo 뒤에 붙을 숫자
    # ent =>EntInfo 뒤에 붙을 숫자
    # mor =>MorInfo 뒤에 붙을 숫자
    sem = 1
    syn = 1
    dom = 1
    ent = 1
    mor = 1

    # x를 컬럼의 개수 만큼의 숫자로 지정해 준다.
    # col_val은 해당 df의 열을 리스트화 시켜준 것이다.
    for x in range (0, l):
        col_val = df.iloc[:, x].tolist ()
        # cnt가 0이면 일치하는 값을 못 찾았다는 의미로 해석(ex 모두 빈칸인 열을 만났을 때)
        # 밑에서 cnt == 0 일때 앞에 정보를 보고 빈칸의 정보를 수정할 때 사용한다.
        cnt = 0
        # k로 col_val의 리스트 요소들을 하나씩 지정해주면서
        # k가 sem_rgx, syn_rgx, dom_rgx, ent_rgx, mor_rgx에 해당되면
        # 컬럼에 일치하는 값이 있었다는 의미로 cnt를 1 증가시켜 주고
        # Info뒤에 붙을 숫자를 1씩 증가시켜 주고
        # 비효율적인 탐색을 막기 위해 바로 break시켜준다.
        for k in col_val:
            if sem_rgx.match (k):
                col_nme[x] = 'SemInfo' + str (sem)
                sem += 1
                cnt += 1
                break

            elif syn_rgx.match (k):
                col_nme[x] = 'SynInfo' + str (syn)
                syn += 1
                cnt += 1
                break

            elif dom_rgx.match (k):
                col_nme[x] = 'DomInfo' + str (dom)
                dom += 1
                cnt += 1
                break

            elif ent_rgx.match (k):
                col_nme[x] = 'EntInfo' + str (ent)
                ent += 1
                cnt += 1
                break

            elif mor_rgx.match (k):
                col_nme[x] = 'MorInfo' + str (mor)
                mor += 1
                cnt += 1
                break

        # 만약 위에서 일치하는 값을 못찾았을 때(ex 모두 빈칸인 열이었을 때)
        # cnt는 0이므로 앞에 col_nme의 정보를 보고
        # 해당 정보와 일치하는 정보의 Info숫자를 증가시켜준 값을 해당 리스트 위치에 저장해준다.
        if cnt == 0:
            if 'Sem' in col_nme[x - 1]:
                col_nme[x] = 'SemInfo' + str (sem)
                sem += 1

            elif 'Syn' in col_nme[x - 1]:
                col_nme[x] = 'SynInfo' + str (syn)
                syn += 1

            elif 'Dom' in col_nme[x - 1]:
                col_nme[x] = 'DomInfo' + str (dom)
                dom += 1

            elif 'Ent' in col_nme[x - 1]:
                col_nme[x] = 'EntInfo' + str (ent)
                ent += 1

            elif 'Mor' in col_nme[x - 1]:
                col_nme[x] = 'MorInfo' + str (mor)
                mor += 1

    df.columns = col_nme

    return df

def Divide(korean_word):
    CHOSUNG_LIST = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
    # 중성 리스트. 00 ~ 20
    JUNGSUNG_LIST = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ',
                     'ㅣ']
    # 종성 리스트. 00 ~ 27 + 1(1개 없음)
    JONGSUNG_LIST = [' ', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ',
                     'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
    r_lst = []
    for w in list(korean_word.strip()):
        ## 영어인 경우 구분해서 작성함.
        if '가'<=w<='힣':
            ## 588개 마다 초성이 바뀜.
            ch1 = (ord(w) - ord('가'))//588
            ## 중성은 총 28가지 종류
            ch2 = ((ord(w) - ord('가')) - (588*ch1)) // 28
            ch3 = (ord(w) - ord('가')) - (588*ch1) - 28*ch2
            r_lst.append([CHOSUNG_LIST[ch1], JUNGSUNG_LIST[ch2], JONGSUNG_LIST[ch3]])
        else:
            r_lst.append([w])
    return r_lst


In [13]:
def merge():
    # 들어오는 데이터를 병합할 변수

    merge_data = pd.read_csv(r"E:\Programming\python\NLP\DecoLexO\DecoLexO\example\Merge1.csv", header=None, encoding='utf-8-sig')
    merge_data = column_name(merge_data)
    data_files = [r'E:\Programming\python\NLP\DecoLexO\DecoLexO\example\Merge2.csv', r"E:\Programming\python\NLP\DecoLexO\DecoLexO\example\Merge3.csv"]
    
    data_df_list = []
    for i in data_files:
        temp_data = pd.read_csv(i, header=None, encoding='utf-8-sig')
        temp_data = column_name(temp_data)
        data_df_list.append(temp_data)
    # 두 번째부터는 들어오는 데이터를 data변수에 저장하고
    # 이전에 저장해둔 merge_data와 data를 concat으로 병합한 뒤
    # 두 데이터를 sort_values로 정렬을 시켜준다.
    for data in data_df_list:
        merge_data = pd.concat ([merge_data, data], ignore_index=True)
    merge_data = merge_data.sort_values (by='Lemma')
    merge_data = merge_data.reset_index ()
    # merge_data = merge_data.dropna(axis=0)

    # 사용자가 입력할 col_name.
    # gui상에서 .text()로 입력받는다.
    col_data = 'MorInfo1'

    # 우선시 되는 데이터 : my_data, 지워져도 되는 데이터 del_data
    # gui상에서 .text()로 입력받는다.
    rules = [['ZNZ','ZNW'],['ZNE','ZNW']]
    for rule in rules:
        my_data = rule[0]
        del_data = rule[1]

        # 중복 단어들을 저장하는 리스트
        reduplication = []
        # i는 처음부터 끝에서 두번째에 있는 단어들까지 돌아가고
        # j는 i번째에 있는 단어 바로 다음 단어랑 비교를 한다.
        # 왜냐하면 정렬이 되어있는 상태기 때문에 바로 전 후의 단어들을 비교해서
        # 앞글자가 다르면 패스를 해서 시간을 단축시키기 위함이다.

        for i in range (0, len(merge_data) - 1):
            for j in range (i + 1, i + 2):
                # first에는 i번째 단어를 음절별로 나누어서 저장하고
                # second에는 j번째(i다음 단어)를 음절로 나누어서 저장한다.
                first = merge_data.loc[i, 'Lemma']
                second = merge_data.loc[j, 'Lemma']
                print(first[0])

                # # 저장한 단어들의 앞글자가 다르면 i를 1증가시켜 다음 단어로 넘어간다.
                # if first[0] != second[0]:
                #     break

                # # 만약 단어의 앞글자가 같다면 아래 코드를 실행한다.
                # elif first[0] == second[0]:
                #     # 앞글자는 같지만 단어가 다르면 for문을 종료한다.
                #     if first != second:
                #         break

                # 앞글자가 같고 단어까지 같으면 입력 받은 mydata가 i번째에 있는지 아님 j(i+1)번째 있는지 확인한다.
                if first == second:
                    if my_data in merge_data.loc[i, col_data] and my_data in merge_data.loc[j, col_data]:
                        pass
                    elif my_data in merge_data.loc[i, col_data] and del_data in merge_data.loc[j, col_data]:
                        # reduplication.append(merge_data.iloc[i].values.tolist())
                        # reduplication.append(merge_data.iloc[j].values.tolist())

                        # stem_data에는 i번째 단어 정보를 리스트형태로 저장하고
                        # follow_data에는 j번째 (i+1)번째 단어 정보를 리스트 형태로 저장한다.
                        # 변수 x는 follow_data를 돌면서 follow_data요소가 stem_data에 정보가 없으면
                        # 그 정보들 check_info()함수에 넘겨서 정보를 col에 저장해준다(ex. MorInfo)
                        # y는 merge_data의 colum 정보들을 돌면서
                        # check_info로 입력받은 정보가 들어있는 column이 처음으로 빈칸이 나오는 장소에
                        # stem_data에 들어있지 않은 정보(follow_data)를 merge_data에 넣어준다.
                        stem_data1 = merge_data.iloc[i].values.tolist ()
                        follow_data1 = merge_data.iloc[j].values.tolist ()
                        for x in range (4, len (follow_data1) - 1):
                            if follow_data1[x] not in stem_data1:
                                print(follow_data1[x])
                                col = check_info (follow_data1[x])
                                col_nme = merge_data.columns
                                for y in range (4, len (col_nme)):
                                    if col in col_nme[y] and merge_data.loc[j, col_nme[y]] == '':
                                        merge_data.loc[j, col_nme[y]] = follow_data1[x]
                                        break

                    elif del_data in merge_data.loc[i, col_data] and my_data in merge_data.loc[j, col_data]:
                        # reduplication.append(merge_data.iloc[i].values.tolist())
                        # reduplication.append(merge_data.iloc[j].values.tolist())
                        stem_data2 = merge_data.iloc[j].values.tolist ()
                        follow_data2 = merge_data.iloc[i].values.tolist ()
                        for x in range (4, len (follow_data2) - 1):
                            if follow_data2[x] not in stem_data2:
                                col = check_info (follow_data2[x])
                                col_nme = merge_data.columns
                                for y in range (4, len (col_nme)):
                                    if col in col_nme[y] and merge_data.loc[j, col_nme[y]] == '':
                                        merge_data.loc[j, col_nme[y]] = follow_data2[x]
                                        break
                else:
                    break
    print(merge_data)

merge()

가
nan


TypeError: expected string or bytes-like object

In [31]:

# 들어오는 데이터를 병합할 변수

merge_data = pd.read_csv(r"E:\Programming\python\NLP\DecoLexO\DecoLexO\example\Merge1.csv", header=None, encoding='utf-8-sig')
merge_data = column_name(merge_data)
data_files = [r'E:\Programming\python\NLP\DecoLexO\DecoLexO\example\Merge2.csv', r"E:\Programming\python\NLP\DecoLexO\DecoLexO\example\Merge3.csv"]

data_df_list = []
for i in data_files:
    temp_data = pd.read_csv(i, header=None, encoding='utf-8-sig')
    temp_data = column_name(temp_data)
    data_df_list.append(temp_data)
# 두 번째부터는 들어오는 데이터를 data변수에 저장하고
# 이전에 저장해둔 merge_data와 data를 concat으로 병합한 뒤
# 두 데이터를 sort_values로 정렬을 시켜준다.
for data in data_df_list:
    merge_data = pd.concat ([merge_data, data], ignore_index=True)
merge_data = merge_data.sort_values (by='Lemma')
merge_data = merge_data.reset_index ()
merge_data = merge_data.fillna('')
del merge_data['index']

# 우선시 되는 데이터 : my_data, 지워져도 되는 데이터 del_data
# gui상에서 .text()로 입력받는다.
rules = [['ZNZ','ZNW'],['ZNE','ZNW']]
for rule in rules:
    my_data = rule[0]
    del_data = rule[1]

    # 중복 단어들을 저장하는 리스트
    reduplication = []
    # i는 처음부터 끝에서 두번째에 있는 단어들까지 돌아가고
    # j는 i번째에 있는 단어 바로 다음 단어랑 비교를 한다.
    # 왜냐하면 정렬이 되어있는 상태기 때문에 바로 전 후의 단어들을 비교해서
    # 앞글자가 다르면 패스를 해서 시간을 단축시키기 위함이다.

    for i in range (0, len(merge_data) - 1):
        for j in range (i + 1, i + 2):
            # first에는 i번째 단어를 음절별로 나누어서 저장하고
            # second에는 j번째(i다음 단어)를 음절로 나누어서 저장한다.
            first = merge_data.loc[i, 'Lemma']
            second = merge_data.loc[j, 'Lemma']

            if first == second:
                if my_data in merge_data.loc[i, 'MorInfo1'] and my_data in merge_data.loc[j, 'MorInfo1']:
                        pass
                if my_data in merge_data.loc[i, 'MorInfo1'] and del_data in merge_data.loc[j, 'MorInfo1']:
                        # reduplication.append(merge_data.iloc[i].values.tolist())
                        # reduplication.append(merge_data.iloc[j].values.tolist())

                        # stem_data에는 i번째 단어 정보를 리스트형태로 저장하고
                        # follow_data에는 j번째 (i+1)번째 단어 정보를 리스트 형태로 저장한다.
                        # 변수 x는 follow_data를 돌면서 follow_data요소가 stem_data에 정보가 없으면
                        # 그 정보들 check_info()함수에 넘겨서 정보를 col에 저장해준다(ex. MorInfo)
                        # y는 merge_data의 colum 정보들을 돌면서
                        # check_info로 입력받은 정보가 들어있는 column이 처음으로 빈칸이 나오는 장소에
                        # stem_data에 들어있지 않은 정보(follow_data)를 merge_data에 넣어준다.
                        stem_data1 = merge_data.iloc[i].values.tolist ()
                        follow_data1 = merge_data.iloc[j].values.tolist ()
                        for x in range (4, len (follow_data1) - 1):
                            if follow_data1[x] not in stem_data1:
                                print(follow_data1[x])
                                col = check_info (follow_data1[x])
                                col_nme = merge_data.columns
                                for y in range (4, len (col_nme)):
                                    if col in col_nme[y] and merge_data.loc[j, col_nme[y]] == '':
                                        merge_data.loc[j, col_nme[y]] = follow_data1[x]
                                        break

                elif del_data in merge_data.loc[i, 'MorInfo1'] and my_data in merge_data.loc[j, 'MorInfo1']:
                        # reduplication.append(merge_data.iloc[i].values.tolist())
                        # reduplication.append(merge_data.iloc[j].values.tolist())
                        stem_data2 = merge_data.iloc[j].values.tolist ()
                        follow_data2 = merge_data.iloc[i].values.tolist ()
                        for x in range (4, len (follow_data2) - 1):
                            if follow_data2[x] not in stem_data2:
                                col = check_info (follow_data2[x])
                                col_nme = merge_data.columns
                                for y in range (4, len (col_nme)):
                                    if col in col_nme[y] and merge_data.loc[j, col_nme[y]] == '':
                                        merge_data.loc[j, col_nme[y]] = follow_data2[x]
                                        break
            else:
                break

print(merge_data)


nan


TypeError: expected string or bytes-like object

In [38]:
def merge():
    # 들어오는 데이터를 병합할 변수

   # 들어오는 데이터를 병합할 변수

    merge_data = pd.read_csv(r"E:\Programming\python\NLP\DecoLexO\DecoLexO\example\Merge1.csv", header=None, encoding='utf-8-sig')
    merge_data = column_name(merge_data)
    data_files = [r'E:\Programming\python\NLP\DecoLexO\DecoLexO\example\Merge2.csv', r"E:\Programming\python\NLP\DecoLexO\DecoLexO\example\Merge3.csv"]

    for i in data_files:
        temp_data = pd.read_csv(i, header=None, encoding='utf-8-sig')
        temp_data = column_name(temp_data)
        data_df_list.append(temp_data)
    # 두 번째부터는 들어오는 데이터를 data변수에 저장하고
    # 이전에 저장해둔 merge_data와 data를 concat으로 병합한 뒤
    # 두 데이터를 sort_values로 정렬을 시켜준다.
    for data in data_df_list:
        merge_data = pd.concat ([merge_data, data], ignore_index=True)
    merge_data = merge_data.sort_values (by='Lemma')
    merge_data = merge_data.reset_index ()
    merge_data = merge_data.fillna('')
    del merge_data['index']
    cnt = 1
    # 사용자가 입력할 col_name.
    # gui상에서 .text()로 입력받는다.
    col_data = 'MorInfo1'

    # 우선시 되는 데이터 : my_data, 지워져도 되는 데이터 del_data
    # gui상에서 .text()로 입력받는다.
    rules = [['ZNZ','ZNW'],['ZNE','ZNW']]
    for rule in rules:
        my_data = rule[0]
        del_data = rule[1]
        for i in range (0, len(merge_data) - 1):
            for j in range (i + 1, i + cnt + 1):
                # first에는 i번째 단어를 음절별로 나누어서 저장하고
                # second에는 j번째(i다음 단어)를 음절로 나누어서 저장한다.
                first = Divide (merge_data.loc[i, 'Lemma'])
                second = Divide (merge_data.loc[j, 'Lemma'])
                if first[0] != second[0]:
                    break
                    # 만약 단어의 앞글자가 같다면 아래 코드를 실행한다.
                else:
                    # 앞글자는 같지만 단어가 다르면 for문을 종료한다.
                    if first != second:
                        break

                    # 앞글자가 같고 단어까지 같으면 입력 받은 mydata가 i번째에 있는지 아님 j(i+1)번째 있는지 확인한다.
                    else:
                        if my_data == merge_data.loc[i, col_data] and del_data == merge_data.loc[j, col_data]:
                            # reduplication.append(merge_data.iloc[i].values.tolist())
                            # reduplication.append(merge_data.iloc[j].values.tolist())

                            # stem_data에는 i번째 단어 정보를 리스트형태로 저장하고
                            # follow_data에는 j번째 (i+1)번째 단어 정보를 리스트 형태로 저장한다.
                            # 변수 x는 follow_data를 돌면서 follow_data요소가 stem_data에 정보가 없으면
                            # 그 정보들 check_info()함수에 넘겨서 정보를 col에 저장해준다(ex. MorInfo)
                            # y는 merge_data의 colum 정보들을 돌면서
                            # check_info로 입력받은 정보가 들어있는 column이 처음으로 빈칸이 나오는 장소에
                            # stem_data에 들어있지 않은 정보(follow_data)를 merge_data에 넣어준다.
                            stem_data1 = merge_data.iloc[i].values.tolist ()
                            follow_data1 = merge_data.iloc[j].values.tolist ()
                            for x in range (4, len (follow_data1) - 1):
                                if follow_data1[x] not in stem_data1:
                                    col = check_info (follow_data1[x])
                                    col_nme = merge_data.columns
                                    for y in range (4, len (col_nme)):
                                        if col in col_nme[y] and merge_data.loc[j, col_nme[y]] == '':
                                            merge_data.loc[j, col_nme[y]] = follow_data1[x]
                                            break

                        elif my_data == merge_data.loc[i, col_data] and del_data == merge_data.loc[j, col_data]:
                            # reduplication.append(merge_data.iloc[i].values.tolist())
                            # reduplication.append(merge_data.iloc[j].values.tolist())
                            stem_data2 = merge_data.iloc[j].values.tolist ()
                            follow_data2 = merge_data.iloc[i].values.tolist ()
                            for x in range (4, len (follow_data2) - 1):
                                if follow_data2[x] not in stem_data2:
                                    col = check_info (follow_data2[x])
                                    col_nme = merge_data.columns
                                    for y in range (4, len (col_nme)):
                                        if col in col_nme[y] and merge_data.loc[j, col_nme[y]] == '':
                                            merge_data.loc[j, col_nme[y]] = follow_data2[x]
                                            break

                        else:
                            pass

                    
        print(merge_data)
        
merge()

Lemma Category MorInfo1 MorInfo2 MorInfo3 MorInfo4 MorInfo5 MorInfo6  \
0       가게     NS01      ZNZ      LEO      SLB      HAL      MCO            
1       가게     NS01      ZNW                                                
2       가게     NS01      ZNW                                                
3       가게     NS01      ZNW                                                
4       가게     NS01      ZNW                                                
...    ...      ...      ...      ...      ...      ...      ...      ...   
4731   지은이     NS01      ZNZ                                                
4732   지은이     NS01      ZNZ                                                
4733   지은이     NS01      ZNZ                                                
4734   지은이     NS01      ZNZ                                                
4735   지은이     NS01      ZNZ                                                

     MorInfo7 MorInfo8  ... DomInfo4 DomInfo5 DomInfo6 DomInfo7 DomInfo8  \
0   

In [56]:
import pandas as pd
import re
import numpy as np

def column_name(df):
    # 첫 행 살리기
    first = list (df.columns)
    if first[0] == 0:
        pass
    else:
        df.loc[0] = first
        for val in first:
            if 'Unnamed' in val:
                x = first.index (val)
                first[x] = np.nan
        df.loc[0] = first
    df = df.fillna ('')
    sem_rgx = re.compile (r'[Q][A-Z]{3}')  # semantic tagset
    syn_rgx = re.compile (r'[Y][A-Z]{3}')  # syntactic tagset
    dom_rgx = re.compile (r'[X]{1}[ABCDEFGHIJKLMNOPQRSTUVWYZ]{3}')  # domain tagset
    ent_rgx = re.compile (r'[X]{2}[A-Z]{2}')  # entity tagset
    mor_rgx = re.compile (r'[A-Z]{3}')  # morph tagset

    # 컬럼의 총 개수를 l에 저장한다.
    # 컬럼의 개수 만큼 lemma와 category뒤에 lemma와 category개수인 2를 뺀만큼
    # ''를 추가해 주어 해당 컬럼 개수 만큼의 리스트 col_nme을 만들어 준다.
    l = len (df.columns)
    col_nme = ['Lemma', 'Category']
    for i in range (l - 2):
        col_nme.append ('')
    # sem =>SemInfo 뒤에 붙을 숫자
    # syn =>SynInfo 뒤에 붙을 숫자
    # dom =>DomInfo 뒤에 붙을 숫자
    # ent =>EntInfo 뒤에 붙을 숫자
    # mor =>MorInfo 뒤에 붙을 숫자
    sem = 1
    syn = 1
    dom = 1
    ent = 1
    mor = 1

    # x를 컬럼의 개수 만큼의 숫자로 지정해 준다.
    # col_val은 해당 df의 열을 리스트화 시켜준 것이다.
    for x in range (0, l):
        col_val = df.iloc[:, x].tolist ()
        # cnt가 0이면 일치하는 값을 못 찾았다는 의미로 해석(ex 모두 빈칸인 열을 만났을 때)
        # 밑에서 cnt == 0 일때 앞에 정보를 보고 빈칸의 정보를 수정할 때 사용한다.
        cnt = 0
        # k로 col_val의 리스트 요소들을 하나씩 지정해주면서
        # k가 sem_rgx, syn_rgx, dom_rgx, ent_rgx, mor_rgx에 해당되면
        # 컬럼에 일치하는 값이 있었다는 의미로 cnt를 1 증가시켜 주고
        # Info뒤에 붙을 숫자를 1씩 증가시켜 주고
        # 비효율적인 탐색을 막기 위해 바로 break시켜준다.
        for k in col_val:
            if sem_rgx.match (k):
                col_nme[x] = 'SemInfo' + str (sem)
                sem += 1
                cnt += 1
                break

            elif syn_rgx.match (k):
                col_nme[x] = 'SynInfo' + str (syn)
                syn += 1
                cnt += 1
                break

            elif dom_rgx.match (k):
                col_nme[x] = 'DomInfo' + str (dom)
                dom += 1
                cnt += 1
                break

            elif ent_rgx.match (k):
                col_nme[x] = 'EntInfo' + str (ent)
                ent += 1
                cnt += 1
                break

            elif mor_rgx.match (k):
                col_nme[x] = 'MorInfo' + str (mor)
                mor += 1
                cnt += 1
                break

        # 만약 위에서 일치하는 값을 못찾았을 때(ex 모두 빈칸인 열이었을 때)
        # cnt는 0이므로 앞에 col_nme의 정보를 보고
        # 해당 정보와 일치하는 정보의 Info숫자를 증가시켜준 값을 해당 리스트 위치에 저장해준다.
        if cnt == 0:
            if 'Sem' in col_nme[x - 1]:
                col_nme[x] = 'SemInfo' + str (sem)
                sem += 1

            elif 'Syn' in col_nme[x - 1]:
                col_nme[x] = 'SynInfo' + str (syn)
                syn += 1

            elif 'Dom' in col_nme[x - 1]:
                col_nme[x] = 'DomInfo' + str (dom)
                dom += 1

            elif 'Ent' in col_nme[x - 1]:
                col_nme[x] = 'EntInfo' + str (ent)
                ent += 1

            elif 'Mor' in col_nme[x - 1]:
                col_nme[x] = 'MorInfo' + str (mor)
                mor += 1

    df.columns = col_nme

    return df
def Divide(korean_word):
    CHOSUNG_LIST = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
    # 중성 리스트. 00 ~ 20
    JUNGSUNG_LIST = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ',
                     'ㅣ']
    # 종성 리스트. 00 ~ 27 + 1(1개 없음)
    JONGSUNG_LIST = [' ', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ',
                     'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
    r_lst = []
    for w in list(korean_word.strip()):
        ## 영어인 경우 구분해서 작성함.
        if '가'<=w<='힣':
            ## 588개 마다 초성이 바뀜.
            ch1 = (ord(w) - ord('가'))//588
            ## 중성은 총 28가지 종류
            ch2 = ((ord(w) - ord('가')) - (588*ch1)) // 28
            ch3 = (ord(w) - ord('가')) - (588*ch1) - 28*ch2
            r_lst.append([CHOSUNG_LIST[ch1], JUNGSUNG_LIST[ch2], JONGSUNG_LIST[ch3]])
        else:
            r_lst.append([w])
    return r_lst
def check_info(word):
    sem_rgx = re.compile (r'[Q][A-Z]{3}')  # semantic tagset
    syn_rgx = re.compile (r'[Y][A-Z]{3}')  # syntactic tagset
    dom_rgx = re.compile (r'[X]{1}[ABCDEFGHIJKLMNOPQRSTUVWYZ]{3}')  # domain tagset
    ent_rgx = re.compile (r'[X]{2}[A-Z]{2}')  # entity tagset
    mor_rgx = re.compile (r'[A-Z]{3}')  # morph tagset

    if sem_rgx.match (word):
        return 'SenInfo'
    elif syn_rgx.match (word):
        return 'SynInfo'
    elif dom_rgx.match (word):
        return 'DomInfo'
    elif ent_rgx.match (word):
        return 'EntInfo'
    elif mor_rgx.match (word):
        return 'MorInfo'
def merge():
    # 들어오는 데이터를 병합할 변수

    merge_data = pd.read_csv(r"E:\Programming\python\NLP\DecoLexO\DecoLexO\example\Merge1.csv", header=None, encoding='utf-8-sig')
    merge_data = column_name(merge_data)
    data_files = [r'E:\Programming\python\NLP\DecoLexO\DecoLexO\example\Merge2.csv', r'E:\Programming\python\NLP\DecoLexO\DecoLexO\example\Merge3.csv']
    data_df_list = []
    for i in data_files:
        temp_data = pd.read_csv(i, header=None, encoding='utf-8-sig')
        temp_data = column_name(temp_data)
        data_df_list.append(temp_data)
    # 두 번째부터는 들어오는 데이터를 data변수에 저장하고
    # 이전에 저장해둔 merge_data와 data를 concat으로 병합한 뒤
    # 두 데이터를 sort_values로 정렬을 시켜준다.
    for data in data_df_list:
        merge_data = pd.concat ([merge_data, data], ignore_index=True)
    merge_data = merge_data.sort_values (by='Lemma')
    merge_data = merge_data.reset_index ()
    merge_data = merge_data.fillna('')
    del merge_data['index']
    cnt = 1
    # 사용자가 입력할 col_name.
    # gui상에서 .text()로 입력받는다.
    col_data = 'MorInfo1'

    # 우선시 되는 데이터 : my_data, 지워져도 되는 데이터 del_data
    # gui상에서 .text()로 입력받는다.
    rules = [['ZNZ','ZNW'],['ZNE','ZNW']]
    del_row_list = []
    for rule in rules:
        my_data = rule[0]
        del_data = rule[1]
        for i in range (0, len(merge_data) - 1):
            for j in range (i + 1, i + cnt + 1):
                # first에는 i번째 단어를 음절별로 나누어서 저장하고
                # second에는 j번째(i다음 단어)를 음절로 나누어서 저장한다.
                first = Divide (merge_data.loc[i, 'Lemma'])
                second = Divide (merge_data.loc[j, 'Lemma'])
                if first[0] != second[0]:
                    break
                    # 만약 단어의 앞글자가 같다면 아래 코드를 실행한다.
                else:
                    # 앞글자는 같지만 단어가 다르면 for문을 종료한다.
                    if first != second:
                        break

                    # 앞글자가 같고 단어까지 같으면 입력 받은 mydata가 i번째에 있는지 아님 j(i+1)번째 있는지 확인한다.
                    else:
                        if my_data == merge_data.loc[i, col_data] and del_data == merge_data.loc[j, col_data]:
                            # reduplication.append(merge_data.iloc[i].values.tolist())
                            # reduplication.append(merge_data.iloc[j].values.tolist())

                            # stem_data에는 i번째 단어 정보를 리스트형태로 저장하고
                            # follow_data에는 j번째 (i+1)번째 단어 정보를 리스트 형태로 저장한다.
                            # 변수 x는 follow_data를 돌면서 follow_data요소가 stem_data에 정보가 없으면
                            # 그 정보들 check_info()함수에 넘겨서 정보를 col에 저장해준다(ex. MorInfo)
                            # y는 merge_data의 colum 정보들을 돌면서
                            # check_info로 입력받은 정보가 들어있는 column이 처음으로 빈칸이 나오는 장소에
                            # stem_data에 들어있지 않은 정보(follow_data)를 merge_data에 넣어준다.
                            stem_data1 = merge_data.iloc[i].values.tolist ()
                            follow_data1 = merge_data.iloc[j].values.tolist ()
                            for x in range (4, len (follow_data1) - 1):
                                if follow_data1[x] not in stem_data1:
                                    col = check_info (follow_data1[x])
                                    col_nme = merge_data.columns
                                    for y in range (4, len(col_nme)):
                                        if col in col_nme[y] and merge_data.loc[j, col_nme[y]] == '':
                                            merge_data.loc[j, col_nme[y]] = follow_data1[x]
                                            break
                            del_row_list.append(j)

                        elif my_data == merge_data.loc[i, col_data] and del_data == merge_data.loc[j, col_data]:
                            # reduplication.append(merge_data.iloc[i].values.tolist())
                            # reduplication.append(merge_data.iloc[j].values.tolist())
                            stem_data2 = merge_data.iloc[j].values.tolist ()
                            follow_data2 = merge_data.iloc[i].values.tolist ()
                            for x in range (4, len (follow_data2) - 1):
                                if follow_data2[x] not in stem_data2:
                                    col = check_info (follow_data2[x])
                                    col_nme = merge_data.columns
                                    for y in range (4, len (col_nme)):
                                        if col in col_nme[y] and merge_data.loc[j, col_nme[y]] == '':
                                            merge_data.loc[j, col_nme[y]] = follow_data2[x]
                                            break
                            del_row_list.append(j)
                        else:
                            pass
    del_temp = []
    for i in del_row_list:
        del_temp.append(merge_data.iloc[i].tolist())
    del_df = pd.DataFrame(del_temp)
    del_df = column_name(del_df)
    del_df.to_csv("res_del_merge.csv", header=False, index=False, na_rep='', encoding='utf-8-sig')


    merge_data = merge_data.drop(del_row_list, 0)
    merge_data = merge_data.reset_index()
    del merge_data["index"]

    return merge_data.to_csv("res_merge.csv", header=False, index=False, na_rep='', encoding='utf-8-sig')
merge()
print('RM')

2389


PermissionError: [Errno 13] Permission denied: 'res_merge.csv'